In [12]:
import pandas as pd
import os


In [23]:
query = '/home/laura/ANALYSIS/VARIANT_CALLING/ABSCESSUS_ATCC/Variants/ALM13614072-1042/snps.all.ivar.tsv'
query_sample = query.split('/')[-2]

variant_folder = '/home/laura/ANALYSIS/VARIANT_CALLING/ABSCESSUS_ATCC/Variants/'
initial_sample = '10183324ABS'
initial = os.path.join(variant_folder + '/' + initial_sample, 'snps.all.ivar.tsv')

output_directory = '/home/laura/IMPORTANTE/Papers/Abscessus/Intrapaciente/1042/Aislado2023/'
file_comparison = query_sample + '_' + initial_sample + '.xlsx'
output_file = os.path.join(output_directory, file_comparison)

coverage_directory = '/home/laura/ANALYSIS/VARIANT_CALLING/ABSCESSUS_ATCC/Stats/Coverage/'
coverage_file = os.path.join(coverage_directory, initial_sample + '.cov')


In [14]:
query_df = pd.read_csv(query, sep='\t')

initial_df = pd.read_csv(initial, sep='\t')


In [15]:
query_df_filt = query_df[(query_df.ALT_DP >= 20) & (query_df.ALT_FREQ < 0.8) & (
    query_df.ALT_FREQ >= 0.2) & (query_df.TYPE == 'snp')].reset_index()

query_df_filt = query_df_filt[['REGION', 'POS', 'REF',
                               'ALT', 'TOTAL_DP', 'REF_FREQ', 'ALT_FREQ', 'OLDVAR']]


In [16]:
initial_df = initial_df[['POS', 'ALT', 'TOTAL_DP', 'ALT_FREQ']]

initial_df = initial_df.rename(columns={'ALT': 'ALT_' + initial_sample,
                                        'TOTAL_DP': 'DP_' + initial_sample, 'ALT_FREQ': 'ALT_FREQ_' + initial_sample})


In [17]:
comparison = query_df_filt.merge(initial_df, how='left', on='POS')


In [18]:
# comparison.to_excel(output_file, index='None')


In [19]:
# Buscar los valores faltantes en la columna 'DP_' + initial_sample
missing_values = comparison[('DP_' + initial_sample)].isna()


In [20]:
# Filtrar las filas con valores faltantes
missing_rows = comparison[missing_values]


In [22]:
def fill_missing_values(comparison, missing_rows, coverage_file, initial_sample, output_file):
    # Leer el archivo de cobertura como un DataFrame
    coverage_df = pd.read_csv(coverage_file, sep='\t', header=None, names=['Genoma', 'POS', 'DP'])
    
    # Crear un diccionario para almacenar los valores de profundidad por posición
    coverage_dict = dict(zip(coverage_df['POS'], coverage_df['DP']))
    
    # Iterar sobre las filas faltantes y extraer los valores correspondientes del 'coverage_dict'
    for index, row in missing_rows.iterrows():
        pos = row['POS']
        if pd.notnull(pos) and pos in coverage_dict:
            coverage_value = coverage_dict[pos]
            comparison.at[index, ('DP_' + initial_sample)] = coverage_value
    
    # Exportar el DataFrame a Excel
    comparison.to_excel(output_file, index=False)


In [ ]:
# Llamar a la función y pasar los DataFrames y los argumentos adicionales
fill_missing_values(comparison, missing_rows, coverage_file, initial_sample, output_file)